In [11]:
import numpy as np
import pandas as pd
import os

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonetic

In [2]:
df=pd.read_excel("NPPES_download.xlsx",dtype=str) # NPPES data

In [64]:
df1=pd.read_excel("Quest_master.xlsx",dtype=str)    # Quest Heirarcy data
df2=pd.read_excel("Dun_Quest.xlsx",dtype=str)       # Dun data  
df3=pd.read_excel("Def_Quest.xlsx",dtype=str)       # Def data

In [65]:
def miss_data(df):
    x = ['column_name','missing_data', 'missing_in_percentage']
    missing_data = pd.DataFrame(columns=x)
    columns = df.columns
    for col in columns:
        icolumn_name = col
        imissing_data = df[col].isnull().sum()
        imissing_in_percentage = (df[col].isnull().sum()/df[col].shape[0])*100

        missing_data.loc[len(missing_data)] = [icolumn_name, imissing_data, imissing_in_percentage]
    print(missing_data)

In [66]:
limitPer = len(df) * .80
df = df.dropna(thresh=limitPer,axis=1)

In [67]:
limitPer = len(df1) * .80
df1 = df1.dropna(thresh=limitPer,axis=1)

In [68]:
limitPer = len(df2) * .80
df2 = df2.dropna(thresh=limitPer,axis=1)

In [69]:
limitPer = len(df3) * .80
df3 = df3.dropna(thresh=limitPer,axis=1)

In [70]:
df2=df2.fillna("NA")

In [71]:
print(df2.shape)
for i in df2.columns:
    print(i, "---> Unique values=", df2[i].nunique(),"|||| NA Values=", sum(pd.isnull(df2[i])), "|||type=", df2[i].dtype)

(41, 11)
TIN ---> Unique values= 41 |||| NA Values= 0 |||type= object
Src ---> Unique values= 1 |||| NA Values= 0 |||type= object
L1 ---> Unique values= 1 |||| NA Values= 0 |||type= object
L2 ---> Unique values= 17 |||| NA Values= 0 |||type= object
L3 ---> Unique values= 17 |||| NA Values= 0 |||type= object
L4 ---> Unique values= 11 |||| NA Values= 0 |||type= object
Name ---> Unique values= 27 |||| NA Values= 0 |||type= object
orig_adr1 ---> Unique values= 37 |||| NA Values= 0 |||type= object
orig_city ---> Unique values= 36 |||| NA Values= 0 |||type= object
orig_state ---> Unique values= 20 |||| NA Values= 0 |||type= object
orig_zip ---> Unique values= 37 |||| NA Values= 0 |||type= object


In [58]:
df.head(2)

,NPI,Name,Is_Org_Subpart,Business_Adr1,Business_City,Business_State,Business_Zip,Business_Phone,Practice_Adr1,Practice_City,Practice_State,Practice_Zip,Practice_Phone,Official_Name_First,Official_Name_Last,Official_Position,Official_Phone,Taxonomy_Code
0,1356732473,PLANNED PARENTHOOD OF NORTHERN NEW ENGLAND,N,128 LAKESIDE AVE STE 301,BURLINGTON,VT,054015906,8024489719,501 PORTLAND ST,ST JOHNSBURY,VT,058192054,8027517821,ANASTASIA,GREER,CREDENTIALING COORDINATOR,8024489719,332900000X
1,1962893099,"NJ ANESTHESIA PARTNERS, LLC",N,PO BOX 7135,COLONIA,NJ,070677135,9083585140,433 HACKENSACK AVE,HACKENSACK,NJ,076016319,2015276800,ROBERT,VILLEGAS,OPERATING MANAGER,9083585140,207L00000X


In [59]:
df1.head(2)

,bic_id,bic_name,ndb_name,src,Name,OtherName,tin,orig_adr1,orig_city,orig_state,orig_zip,orig_zip5,orig_zip4
0,NaN,NaN,NaN,D&B,Unilab Corporation,Quest Diagnostics,954415490,8401 Fallbrook Ave,West Hills,CA,913043226,91304,3226
1,NaN,NaN,QUEST DIAGNOSTICS,D&B,"Labone, Inc.","ExamOne, Quest Diagnostics",480952323,10101 Renner Blvd,Lenexa,KS,662199752,66219,9752


In [60]:
df2.head(2)

,TIN,Src,L1,L2,L3,L4,Name,orig_adr1,orig_city,orig_state,orig_zip
0,161387862,Dun,Quest Diagnostics Incorporated,NA,NA,NA,Quest Diagnostics Incorporated,500 Plaza Dr Ste G,Secaucus,NJ,07094-3656
1,954415490,Dun,Quest Diagnostics Incorporated,Unilab Corporation,NA,NA,Unilab Corporation,8401 Fallbrook Ave,West Hills,CA,91304-3226


In [61]:
df3.head(2)

,NPI_NUMBER,HOSPITAL_ID,HOSPITAL_NAME,FIRM_TYPE,COMPANY_STATUS,HQ_ADDRESS,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,HQ_COUNTY,HQ_PHONE,HQ_LATITUDE,HQ_LONGITUDE,NETWORK_ID,NETWORK_PARENT_ID,HOSPITAL_PARENT_ID,SF_PARENT_ACCOUNT_ID
0,1972961415,61277,Governors Ridge,Assisted Living Facility,Active,300 E Devereaux St,Decatur,TX,76234,TX - Wise,9406271104,33,-98,36038,0,0,36038
1,1720311665,75407,Brookdale Tequesta - 205 Village Blvd,Assisted Living Facility,Active,205 Village Blvd,Tequesta,FL,33469,FL - Palm Beach,5617483611,27,-80,581990,0,0,581990


In [62]:
def fuzz_match(col_name1, col_name2, cutoff):
    # col_name1: supposed to be matched against col_name2
    # cutoff:  used as a benchmark(score out of 100)
    # Making sure col_name1 do not have None value
    
    col_name1 = col_name1.fillna('None')

        
        
    list2 = []
    for i in col_name1:
        match_res2 = process.extractOne(i, col_name2, scorer = fuzz.ratio, score_cutoff=cutoff)
        list2.append(match_res2)
    # Using two fuzz method to improve accuracy
    
    lists = [[] for i in range(4)]


    for i in list2:
        if i==None:
            lists[2].append(None)
            lists[3].append(None)
        else:
            lists[2].append(i[0])
            lists[3].append(i[1])
            
    match = pd.DataFrame()
    match['origin'] = col_name1.reset_index(drop = True) # Pay attention to the index of col_name1
    match['ratio_match'] = pd.Series(lists[2])
    match['score2'] = pd.Series(lists[3])
    return match

In [18]:
output1 = fuzz_match(col_name1 = df['Name'], col_name2 = df3['HOSPITAL_NAME'], cutoff=85)
output2 = fuzz_match(col_name1 = df['Business_Adr1'], col_name2 = df3['HQ_ADDRESS'], cutoff=70)
output3 = fuzz_match(col_name1 = df['Business_City'], col_name2 = df3['HQ_CITY'], cutoff=70)
output4 = fuzz_match(col_name1 = df['Business_State'], col_name2 = df3['HQ_STATE'], cutoff=70)
output5 = fuzz_match(col_name1 = df['Business_Zip'], col_name2 = df3['HQ_ZIP_CODE'], cutoff=70)
output6 = fuzz_match(col_name1 = df['NPI'], col_name2 = df3['NPI_NUMBER'], cutoff=85)
#output7 = fuzz_match(col_name1 = df2['Billing_Adr1'], col_name2 = df3['HQ_ADDRESS'], cutoff=70)
#output8 = fuzz_match(col_name1 = df2['Billing_City'], col_name2 = df3['HQ_CITY'], cutoff=70)
#output9 = fuzz_match(col_name1 = df2['Billing_State'], col_name2 = df3['HQ_STATE'], cutoff=70)
#output10 = fuzz_match(col_name1 = df2['Billing_Zip'], col_name2 = df3['HQ_ZIP_CODE'], cutoff=70)

In [72]:
output12 = fuzz_match(col_name1 = df['Business_Phone'], col_name2 = df3['HQ_PHONE'], cutoff=85)

TypeError: expected string or bytes-like object

In [36]:
output7 = fuzz_match(col_name1 = df['Name'], col_name2 = df1['Name'], cutoff=85)
#output8= fuzz_match(col_name1 = df['Business_Adr1'], col_name2 = df1['orig_adr1'], cutoff=70)


TypeError: expected string or bytes-like object

In [28]:
frames = [df,output6, output1, output2, output3, output4, output5, output7, output8,output9,output10, output11]
output = pd.concat(frames, axis=1)

In [29]:
df.columns

Index(['NPI', 'Name', 'Is_Org_Subpart', 'Business_Adr1', 'Business_City',
       'Business_State', 'Business_Zip', 'Business_Phone', 'Practice_Adr1',
       'Practice_City', 'Practice_State', 'Practice_Zip', 'Practice_Phone',
       'Official_Name_First', 'Official_Name_Last', 'Official_Position',
       'Official_Phone', 'Taxonomy_Code'],
      dtype='object')

In [27]:
output.head(2)

,TIN,Src,L1,L2,L3,L4,Name,orig_adr1,orig_city,orig_state,...,score2,origin,ratio_match,score2,origin,ratio_match,score2,origin,ratio_match,score2
0,161387862,Dun,Quest Diagnostics Incorporated,NA,NA,NA,Quest Diagnostics Incorporated,500 Plaza Dr Ste G,Secaucus,NJ,...,NaN,BURLINGTON,Bloomington,76.0,VT,None,NaN,054015906,None,NaN
1,954415490,Dun,Quest Diagnostics Incorporated,Unilab Corporation,NA,NA,Unilab Corporation,8401 Fallbrook Ave,West Hills,CA,...,NaN,COLONIA,Livonia,71.0,NJ,NJ,100.0,070677135,None,NaN


In [32]:
output.columns = ['NPI', 'Name', 'Is_Org_Subpart', 'Business_Adr1', 'Business_City',
       'Business_State', 'Business_Zip', 'Business_Phone', 'Practice_Adr1',
       'Practice_City', 'Practice_State', 'Practice_Zip', 'Practice_Phone',
       'Official_Name_First', 'Official_Name_Last', 'Official_Position',
       'Official_Phone', 'Taxonomy_Code',
                  'NPI', 'defNPI', 'NPIScore',
                  'Name', 'DefName','DefNameScore', 
                  'Address','DefAddress', 'DefAdrsScore', 
                  'City', 'DefCity', 'DefCityScore',
                  'State', 'DefState', 'DefStateScore',
                  'Zip', 'DefZip', 'DefZipScore']

In [33]:
#output["WebScore"]=output["WebNameScore"]+output["WebAdrsScore"]+output["WebCityScore"]+output['WebStateScore']+output["WebZipScore"]

output["DefScore"]=output["NPIScore"]+output["DefNameScore"]+output["DefAdrsScore"]+output["DefCityScore"]+output['DefStateScore']+output["DefZipScore"]

In [34]:
output.to_excel("Fuzzy_Quest_Final.xlsx")